# Estrutura da Rede Neural
#### Criando uma Rede Neural para analisar a venda de jogos em diferentes países usando regressão.

### Importando as Bibliotecas

In [1]:
# importando a biblioteca pandas do python
import pandas as pd
# importando da biblioteca sklearn do python
from sklearn.preprocessing import (LabelEncoder, 
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
# importando da biblioteca keras do python
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

Using TensorFlow backend.


### Tratando os Dados

In [3]:
# importando os dados no objeto 'dataframe'
dataframe = pd.read_csv('games.csv', 
                        encoding = 'utf-8',
                       sep = ',')

In [4]:
# visualizando o dataframe
dataframe

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# descartando algumas colunas de pouca relevância 
# para a rede neural
dataframe = dataframe.drop('Other_Sales', axis = 1)
dataframe = dataframe.drop('Global_Sales', axis = 1)
dataframe = dataframe.drop('Developer', axis = 1)

In [6]:
# visualizando o dataframe
dataframe

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN


In [7]:
# visualizando a dimensão do dataframe
print(dataframe.shape)

(16719, 13)


In [8]:
# removendo valores faltantes do dataframe
dataframe = dataframe.dropna(axis = 0)

In [9]:
# visualizando a dimensão do dataframe
print(dataframe.shape)

(6825, 13)


In [10]:
# ajustando valores inconsistentes
dataframe = dataframe.loc[dataframe['NA_Sales'] > 1]

In [11]:
# ajustando valores inconsistentes
dataframe = dataframe.loc[dataframe['EU_Sales'] > 1]

In [12]:
# visualizando a dimensão do dataframe
print(dataframe.shape)

(258, 13)


In [13]:
# visualizando os nomes dos jogos
dataframe['Name'].value_counts()
# como não há um padrão especifico, essa coluna será
# removida por não ser relevante para o processamento da
# rede neural

Grand Theft Auto V                 4
Call of Duty: Ghosts               3
The Elder Scrolls V: Skyrim        3
Battlefield 4                      3
Assassin's Creed IV: Black Flag    3
                                  ..
Metal Gear Solid                   1
Call of Duty: Finest Hour          1
Driver                             1
Carnival Games                     1
Kinect Adventures!                 1
Name: Name, Length: 223, dtype: int64

In [14]:
# salvando os nomes dos jogos
nome_jogos = dataframe.Name

In [15]:
# os nomes salvos servem para posterior análise dos 
# resultados previstos pela rede neural
print(nome_jogos)

0                      Wii Sports
2                  Mario Kart Wii
3               Wii Sports Resort
6           New Super Mario Bros.
7                        Wii Play
                  ...            
568                Medal of Honor
573    Tom Clancy's Splinter Cell
591        Mario Strikers Charged
610                    Crazy Taxi
636         The Sims: Bustin' Out
Name: Name, Length: 258, dtype: object


In [16]:
# descartando algumas colunas de pouca relevância 
# para a rede neural
dataframe = dataframe.drop('Name', axis = 1)

In [17]:
# separando os atributos previsores do dataframe
previsores = dataframe.iloc[:, [0, 1, 2, 3, 
                                7, 8, 9, 10, 11]].values

In [18]:
# separando os atributos meta do dataframe
# 1 - vendas na américa do norte
# 2 - vendas na europa
# 3 - vendas no japão
venda_na = dataframe.iloc[:, 4].values
venda_eu = dataframe.iloc[:, 5].values
venda_jp = dataframe.iloc[:, 6].values

In [19]:
# criando o objeto 'labelencoder'
labelencoder = LabelEncoder()

In [20]:
# escalonando os atributos categóricos em numéricos
previsores[:, 0] = labelencoder.fit_transform(previsores[:, 0])
previsores[:, 2] = labelencoder.fit_transform(previsores[:, 2])
previsores[:, 3] = labelencoder.fit_transform(previsores[:, 3])
previsores[:, 8] = labelencoder.fit_transform(previsores[:, 8])

In [21]:
# transformando as variáveis numéricas em variáveis do tipo dummy
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), 
                                                  [0, 2, 3, 8])], 
                                  remainder='passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

In [22]:
# visualizando a dimensão do dataframe
print(previsores.shape)

(258, 61)


In [25]:
# visualizando a dimensão do dataframe
print(venda_na.shape)

(258,)


In [24]:
# visualizando a dimensão do dataframe
print(venda_eu.shape)

(258,)


In [26]:
# visualizando a dimensão do dataframe
print(venda_jp.shape)

(258,)


### Configurando a Rede Neural

In [27]:
# definindo a camada de entrada
camada_entrada = Input(shape=(61,))
# definindo a primeira camada oculta que recebe a camada de entrada
camada_oculta1 = Dense(units = 32, activation = 'sigmoid')(camada_entrada)
# definindo a segunda camada oculta que recebe a primeira camada oculta
camada_oculta2 = Dense(units = 32, activation = 'sigmoid')(camada_oculta1)
# definindo a primeira camada de saída que recebe a segunda camada oculta
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
# definindo a segunda camada de saída que recebe a segunda camada oculta
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
# definindo a terceira camada de saída que recebe a segunda camada oculta
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

In [28]:
# criando o modelo da rede neural
regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1, camada_saida2, 
                             camada_saida3])

In [29]:
# compilando a rede neural
regressor.compile(optimizer = 'adam',
                  loss = 'mse')

### Treinando a Rede Neural

In [30]:
# treinando a rede neural
regressor.fit(previsores, [venda_na, venda_eu, venda_jp],
              epochs = 10000, batch_size = 100, verbose = False)

### Observando as Previsões Feitas

In [31]:
# realizando a previsão para cada uma das camadas de saída
(previsao_na, 
 previsao_eu, 
 previsao_jp) = regressor.predict(previsores)

In [32]:
# visualizando as previsões obtidas de vendas na América do Norte
print(previsao_na)

[[39.92565  ]
 [ 4.5016975]
 [16.916552 ]
 [ 8.280321 ]
 [ 3.4628704]
 [ 4.495038 ]
 [ 9.069305 ]
 [ 7.729294 ]
 [12.995725 ]
 [ 2.835338 ]
 [ 4.9850636]
 [ 2.429373 ]
 [ 4.9850636]
 [ 4.299267 ]
 [ 2.2914958]
 [ 5.2787876]
 [ 3.2517815]
 [ 3.3794959]
 [ 3.8594644]
 [ 3.379294 ]
 [ 3.14696  ]
 [ 3.7339408]
 [ 3.8019955]
 [ 3.3794959]
 [ 4.499902 ]
 [ 3.3680217]
 [ 3.1469579]
 [ 3.8815832]
 [ 4.9850636]
 [ 3.3794255]
 [ 3.146958 ]
 [ 2.3476107]
 [ 4.501697 ]
 [ 3.3794947]
 [ 3.379494 ]
 [ 3.1469579]
 [ 3.379023 ]
 [ 2.2894864]
 [ 3.1692853]
 [ 3.3794959]
 [ 3.3794956]
 [ 4.9850626]
 [ 2.2941818]
 [ 2.2895029]
 [ 3.367967 ]
 [ 3.3794956]
 [ 4.430441 ]
 [ 2.8885243]
 [ 4.9850636]
 [ 4.50142  ]
 [ 3.3794959]
 [ 2.368776 ]
 [ 6.239296 ]
 [ 2.284193 ]
 [ 3.3794444]
 [ 2.5435243]
 [ 2.4360986]
 [ 2.152494 ]
 [ 3.3794954]
 [ 3.3794959]
 [ 3.2813146]
 [ 3.3792093]
 [ 2.2942572]
 [ 2.3007061]
 [ 3.3794959]
 [ 4.9850636]
 [ 2.415325 ]
 [ 3.3794959]
 [ 3.1856267]
 [ 3.3794959]
 [ 3.6110313]
 [ 2.2

In [33]:
# visualizando as previsões obtidas de vendas na Europa
print(previsao_eu)

[[27.35559  ]
 [ 3.2318354]
 [11.58662  ]
 [ 5.567748 ]
 [ 2.5663643]
 [ 3.2272668]
 [ 6.0927854]
 [ 5.3884797]
 [ 8.92761  ]
 [ 2.1564665]
 [ 3.492406 ]
 [ 1.892215 ]
 [ 3.492406 ]
 [ 3.091425 ]
 [ 1.8026443]
 [ 3.7608879]
 [ 2.8544896]
 [ 2.1133306]
 [ 2.7702026]
 [ 2.1131337]
 [ 1.8877697]
 [ 2.9476779]
 [ 2.7259355]
 [ 2.1133306]
 [ 3.2306042]
 [ 2.1032617]
 [ 1.8877678]
 [ 2.840752 ]
 [ 3.492406 ]
 [ 2.1132615]
 [ 1.8877676]
 [ 1.8390872]
 [ 3.231835 ]
 [ 2.1133294]
 [ 2.1133292]
 [ 1.8877678]
 [ 2.1128685]
 [ 1.8013391]
 [ 2.24371  ]
 [ 2.1133306]
 [ 2.1133304]
 [ 3.4924047]
 [ 1.8043885]
 [ 1.8013499]
 [ 2.1032133]
 [ 2.1133304]
 [ 3.1827886]
 [ 2.1911414]
 [ 3.492406 ]
 [ 3.2316446]
 [ 2.1133306]
 [ 1.8528373]
 [ 4.39697  ]
 [ 1.7975912]
 [ 2.1132853]
 [ 1.8811014]
 [ 1.8965864]
 [ 1.6272247]
 [ 2.1133301]
 [ 2.1133306]
 [ 2.302168 ]
 [ 2.113051 ]
 [ 1.8044372]
 [ 1.808625 ]
 [ 2.1133306]
 [ 3.492406 ]
 [ 1.8829417]
 [ 2.1133306]
 [ 2.2538903]
 [ 2.1133306]
 [ 2.5909085]
 [ 1.8

In [34]:
# visualizando as previsões obtidas de vendas no Japão
print(previsao_jp)

[[ 4.5976777 ]
 [ 1.0866932 ]
 [ 2.4760025 ]
 [ 2.7272952 ]
 [ 0.69706076]
 [ 1.0841764 ]
 [ 2.8626251 ]
 [ 1.3633778 ]
 [ 2.0379567 ]
 [ 0.5850068 ]
 [ 0.42734063]
 [ 0.50998056]
 [ 0.42734063]
 [ 1.0077175 ]
 [ 0.48401213]
 [ 0.9985213 ]
 [ 0.09288941]
 [ 0.42099738]
 [ 0.8133964 ]
 [ 0.4209255 ]
 [ 0.3372286 ]
 [ 0.26060104]
 [ 0.7851476 ]
 [ 0.42099738]
 [ 1.0860153 ]
 [ 0.4173031 ]
 [ 0.33722794]
 [ 0.7694014 ]
 [ 0.42734063]
 [ 0.42097253]
 [ 0.33722812]
 [ 0.49461144]
 [ 1.086693  ]
 [ 0.42099702]
 [ 0.42099702]
 [ 0.33722794]
 [ 0.4208293 ]
 [ 0.4836319 ]
 [ 1.4346763 ]
 [ 0.42099738]
 [ 0.42099732]
 [ 0.42734116]
 [ 0.48452032]
 [ 0.48363525]
 [ 0.41728556]
 [ 0.42099732]
 [ 1.0594903 ]
 [ 0.59468305]
 [ 0.42734063]
 [ 1.0865884 ]
 [ 0.42099738]
 [ 0.4985987 ]
 [ 1.1465333 ]
 [ 0.4835369 ]
 [ 0.4209811 ]
 [ 0.37379062]
 [ 0.5112409 ]
 [ 0.90814435]
 [ 0.42099708]
 [ 0.42099738]
 [ 0.5055704 ]
 [ 0.42089558]
 [ 0.48453468]
 [ 0.48575503]
 [ 0.42099738]
 [ 0.42734063]
 [ 0.50776

### Alguma Dúvida? Entre em Contato Comigo:

- [Me envie um e-mail](mailto:alysson.barbosa@ee.ufcg.edu.br);